In [79]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import pprint

In [3]:
candidate_items = pd.read_csv('../../Data/dressipi_recsys2022/candidate_items.csv')
item_features = pd.read_csv('../../Data/dressipi_recsys2022/item_features.csv')


train_purchases = pd.read_csv('../../Data/dressipi_recsys2022/train_purchases.csv')
train_sessions = pd.read_csv('../../Data/dressipi_recsys2022/train_sessions.csv')

In [14]:
items_ids = list(item_features.item_id.unique())
items_ids.sort()
print(items_ids[:10])
print(len(items_ids))

[2, 3, 4, 7, 8, 9, 10, 11, 13, 14]
23691


In [15]:
feature_category_ids = list(item_features.feature_category_id.unique())
feature_category_ids.sort()
print(feature_category_ids)
print(len(feature_category_ids))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73]
73


In [32]:
list(item_features.loc[item_features.feature_category_id == 3].sort_values(by='feature_value_id', ascending=True).feature_value_id.unique())

[241, 569, 645, 793, 889]

# Lista de indice del vector onehot que va a codificar los items

In [44]:
index_vector_onehot = []

for feature_category_id in feature_category_ids:
    
    #print(feature_category_id)
    
    posible_values_feature_category = list(item_features.loc[item_features.feature_category_id == feature_category_id].sort_values(by='feature_value_id', ascending=True).feature_value_id.unique())
    
    #print(posible_values_feature_category)
    
    for posible_value_feature_category in posible_values_feature_category:
        
        index_vector_onehot.append(str(feature_category_id) + '_' + str(posible_value_feature_category))

In [45]:
print(index_vector_onehot[:20])
print(len(index_vector_onehot))

['1_60', '1_143', '1_358', '1_461', '1_517', '1_771', '2_53', '2_56', '2_83', '2_104', '2_121', '2_139', '2_198', '2_208', '2_297', '2_309', '2_310', '2_343', '2_406', '2_442']
904


# Creacion del dataframe de salida

In [46]:
columns_names = index_vector_onehot
columns_names.insert(0, "item_id")
columns_names[:10]

['item_id',
 '1_60',
 '1_143',
 '1_358',
 '1_461',
 '1_517',
 '1_771',
 '2_53',
 '2_56',
 '2_83']

In [47]:
items_onehot_encoded_df = pd.DataFrame(columns = columns_names)
items_onehot_encoded_df

,item_id,1_60,1_143,1_358,1_461,1_517,1_771,2_53,2_56,2_83,...,72_665,72_671,72_712,72_747,72_748,72_751,72_829,72_883,73_91,73_544


In [87]:
for item_id in tqdm(items_ids):
        
    #print('item id')
    #print(item_id)
    
    
    feature_category_values_item_id = item_features.loc[item_features.item_id == item_id].sort_values(['feature_category_id', 'feature_value_id'], ascending=[True,True]).drop('item_id', axis=1)
    
    #print('features y valores')
    #print(feature_category_values_item_id)
    #print('*'*50)
    
    
    dict_item_id_encoded = {'item_id': item_id}

    
    for feature_category_id in feature_category_ids:
        
        #print('-'*30)
        #print(feature_category_id)

        posible_values_feature_category = list(item_features.loc[item_features.feature_category_id == feature_category_id].sort_values(by='feature_value_id', ascending=True).feature_value_id.unique())

        #print(posible_values_feature_category)
        #print('-'*30)
        
        
        values_feature_category_of_item_id = list(feature_category_values_item_id.loc[feature_category_values_item_id.feature_category_id == feature_category_id].feature_value_id)
        #print(values_feature_category_of_item_id)
        
        
        if values_feature_category_of_item_id:
            
            #print('SI hubo')
            
            for posible_value_feature_category in posible_values_feature_category:

                column_index = str(feature_category_id) + '_' + str(posible_value_feature_category)
                
                if posible_value_feature_category in values_feature_category_of_item_id:
                    
                    dict_item_id_encoded[column_index] = 1
                
                else:
                    
                    dict_item_id_encoded[column_index] = 0
            
            
            
        else:
            
            #print('No hubo')
            
            for posible_value_feature_category in posible_values_feature_category:

                column_index = str(feature_category_id) + '_' + str(posible_value_feature_category)
                
                dict_item_id_encoded[column_index] = 0
        
        
        #print('*'*40)
        
        
    items_onehot_encoded_df = items_onehot_encoded_df.append(dict_item_id_encoded, ignore_index = True)
    

    

100%|██████████| 23691/23691 [48:01<00:00,  8.22it/s]


In [93]:
items_onehot_encoded_df

,item_id,1_60,1_143,1_358,1_461,1_517,1_771,2_53,2_56,2_83,...,72_665,72_671,72_712,72_747,72_748,72_751,72_829,72_883,73_91,73_544
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23687,28139,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
23688,28140,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
23689,28141,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
23690,28142,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [94]:
items_onehot_encoded_df.to_csv('../../Data/items_onehot_encoded.csv', index=False)